# MVP Profitability & Monitoring Walkthrough (NN Plus Ultra)

This notebook executes the minimal NN Plus Ultra pipeline so we can iterate on the MVP.
The steps below mirror the optimisation checklist and ensure we capture artefacts for
training, inference, profitability summaries, and live monitoring diagnostics without
dropping any items from the plan.

**Pipeline overview**

1. Generate (or reuse) the SQLite fixture that powers the MVP experiments.
2. Load the quickstart experiment configuration with CPU-friendly overrides.
3. Run the supervised training loop and collect logged metrics.
4. Execute the inference agent to produce predictions and realised-return metrics.
5. Compile the operations summary to surface profitability and guardrail diagnostics.
6. Initialise the live monitor to track rolling risk/drift alerts over new returns.
7. Record notes so we can update the optimisation log without losing any steps.

In [ ]:
from pathlib import Path

import pandas as pd

from market_nn_plus_ultra.automation import (
    DEFAULT_FIXTURE_CONFIG,
    MVPPipelineState,
    build_monitor,
    ensure_fixture,
    extract_reference_returns,
    load_mvp_experiment,
    run_mvp_inference,
    run_mvp_training,
    summarise_operations,
    update_monitor,
)
from market_nn_plus_ultra.evaluation import OperationsThresholds

## 1. Establish paths and initialise run notes

In [ ]:
project_root = Path.cwd().resolve().parent
(data_dir := project_root / "data").mkdir(parents=True, exist_ok=True)

fixture_path = data_dir / "mvp_fixture.db"
config_path = project_root / "configs" / "mvp_quickstart.yaml"

run_notes: list[str] = []
print(f"Project root: {project_root}")
print(f"Fixture path: {fixture_path}")
print(f"Config path: {config_path}")

## 2. Build or reuse the MVP SQLite fixture

In [ ]:
fixture = ensure_fixture(fixture_path, config=DEFAULT_FIXTURE_CONFIG)
run_notes.append(f"Fixture ready at {fixture}")
fixture

## 3. Load the experiment configuration with MVP overrides

In [ ]:
experiment = load_mvp_experiment(
    config_path,
    max_epochs=1,
    limit_train_batches=0.1,
    limit_val_batches=0.2,
    batch_size=8,
    accelerator="cpu",
    devices=1,
)
experiment.data.sqlite_path = fixture
state = MVPPipelineState(experiment_config=experiment)
run_notes.append("Loaded experiment config with CPU-friendly overrides")

{
    "data_path": str(experiment.data.sqlite_path),
    "batch_size": experiment.trainer.batch_size,
    "max_epochs": experiment.trainer.max_epochs,
    "limit_train_batches": experiment.trainer.limit_train_batches,
    "limit_val_batches": experiment.trainer.limit_val_batches,
}

## 4. Run supervised training

In [ ]:
training_result = run_mvp_training(experiment)
state.training = training_result
run_notes.append("Training completed successfully")

{
    "best_model_path": training_result.best_model_path,
    "dataset_summary": training_result.dataset_summary,
    "logged_metrics": training_result.logged_metrics,
    "profitability_summary": training_result.profitability_summary,
}

## 5. Execute the inference agent

In [ ]:
checkpoint_path = Path(training_result.best_model_path) if training_result.best_model_path else None
agent_result = run_mvp_inference(
    experiment,
    checkpoint_path=checkpoint_path,
    device="cpu",
    evaluate=True,
)
state.agent = agent_result
run_notes.append("Inference completed with evaluation metrics")

print("Predictions sample:")
display(agent_result.predictions.head())
print("
Evaluation metrics:")
agent_result.metrics

## 6. Compile profitability and guardrail diagnostics

In [ ]:
ops_thresholds = OperationsThresholds(min_sharpe=0.5, max_drawdown=0.25)
ops_summary = summarise_operations(agent_result.predictions, thresholds=ops_thresholds)
state.operations = ops_summary
run_notes.append("Operations summary compiled with guardrail thresholds")

ops_summary.as_dict()

## 7. Initialise live monitoring

In [ ]:
reference_returns = extract_reference_returns(agent_result.predictions)
monitor = build_monitor(
    reference_returns,
    window_size=128,
    drift_bins=20,
    risk_thresholds=ops_thresholds,
)
state.monitor = monitor
snapshot = update_monitor(monitor, reference_returns[-256:])
state.monitoring_snapshot = snapshot
run_notes.append(f"Monitoring snapshot generated for {snapshot.window_count} returns")

snapshot.as_dict()

## 8. Collate run notes for the optimisation tracker

In [ ]:
run_notes